In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
import imblearn
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import RandomOverSampler

In [52]:
df = pd.read_csv('train.csv')
Label=df['Label']
#oversampling
oversample = RandomOverSampler(sampling_strategy=0.5)
df,y_resempled=oversample.fit_resample(df,Label)
train_end_idx = len(df) 
df_test = pd.read_csv('test.csv')
df_test['Label'] = np.zeros((len(df_test),))

# 為了處理方便，把 'train.csv' 和 'test.csv' 合併起來，'test.csv'的 RainToday 欄位用 0 補起來。
# 以 train_end_idx 作為 'train.csv' 和 'test.csv' 分界列，
df = pd.concat([df, df_test], sort=False)
#pd.concat為合併兩DataFrame
df.drop(["Date","Loc","DayWindDir","NightWindDir"],inplace=True,axis=1)
print(df.shape)

(82140, 18)


In [53]:
Label=df['Label']
df.drop(['Label'],inplace=True,axis=1)
print(len(Label))
col=df.columns

82140


In [54]:
from category_encoders.target_encoder import TargetEncoder
enc=TargetEncoder()
df['WindDir']=enc.fit_transform(df['WindDir'],df['WindSpeed'])

In [55]:
#透過sklearn補值
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=1, random_state=0)
df=imp.fit_transform(df)
df=pd.DataFrame(df,columns=col)

/Users/wuguanzhi/opt/anaconda3/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [56]:
#處理異常值概貌法可用
def cap(df,quantile=[0.01,0.99]):
    Q01,Q99=df.quantile(quantile).values.tolist()
        # 替换异常值为指定的分位数
    if Q01 > df.min():
        df.loc[df <=Q01] = Q01  
    if Q99 < df.max():
        df.loc[df >= Q99] = Q99
for i in df:
    cap(df[i])

In [57]:
# 數據歸一化
from sklearn.preprocessing import StandardScaler
df = pd.DataFrame(StandardScaler().fit_transform(df))
df.columns=col
Label = Label.reset_index()
df=pd.concat([df, Label], axis=1)
df.drop(["index"],inplace=True,axis=1)
df.groupby('Label').count()

,TempLow,TempHigh,Evaporation,Sun,WindDir,WindSpeed,WindSpeed9am,WindSpeed3pm,DayHumidity,NightHumidity,Pressure9am,Pressure3pm,DayCloud,NightCloud,Temp9am,Temp3pm,RISK_MM
Label,,,,,,,,,,,,,,,,,
0.0,66375,66375,66375,66375,66375,66375,66375,66375,66375,66375,66375,66375,66375,66375,66375,66375,66375
1.0,15765,15765,15765,15765,15765,15765,15765,15765,15765,15765,15765,15765,15765,15765,15765,15765,15765


In [58]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns = ['Label']).values[:train_end_idx, :],
    df['Label'].values[:train_end_idx], test_size=0.2)
X_ans = df.drop(columns = ['Label']).values[train_end_idx:, :]


X_train,y_train=shuffle(X_train,y_train)

 
print('----info----')
print('training data size:', len(X_train))
print('testing data size:', len(X_test))

----info----
training data size: 37836
testing data size: 9460


In [59]:
# model 2:linear SVM

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import cross_val_score
svm_linear = SVC(kernel='linear', random_state=0,max_iter=2100,probability=True)

#clf = LinearSVC(random_state=0, tol=1e-5,C=2)

svm_linear.fit(X_train, y_train)

y_pred_linear = svm_linear.predict(X_test)

#y_predprob_linear = svm_linear.predict_proba(X_test)
#y_predprob_po_linear =  y_predprob_linear[:,1]

print('Accuracy: %f' % accuracy_score(y_test, y_pred_linear))
print('f1-score: %f' % f1_score(y_test, y_pred_linear))

/Users/wuguanzhi/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=2100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy: 0.594292
f1-score: 0.454054


In [60]:
def selectFeature(col,coef_):
    min_list=[]
    for i in range(len(col)):
#         if abs(coef_[i])<0.015:
#             min_list.append([col[i],abs(coef_[i])])
        if len(min_list)<5:
            min_list.append([col[i],abs(coef_[i])])
            min_list.sort(reverse=True)   
        else:
            if abs(svm_linear.coef_.ravel()[i])<min_list[-1][1]:  
                min_list[-1]=[col[i],abs(coef_[i])]
                min_list=sorted(min_list,key=lambda min_list:min_list[1])
    return min_list

for i in selectFeature(df.columns.drop('Label'),svm_linear.coef_.ravel()):
    df.drop([i[0]],inplace=True,axis=1)
    print('%s has been dropped， it coef '%i[0])

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns = ['Label']).values[:train_end_idx, :],
    df['Label'].values[:train_end_idx], test_size=0.25)
X_ans = df.drop(columns = ['Label']).values[train_end_idx:, :]


X_train,y_train=shuffle(X_train,y_train)
dbscan

WindDir has been dropped， it coef 
TempHigh has been dropped， it coef 
NightCloud has been dropped， it coef 
Temp3pm has been dropped， it coef 
DayCloud has been dropped， it coef 


In [61]:
from sklearn.decomposition import PCA
print("before",df.shape)
pca=PCA(n_components=2)
df=pca.fit(df).transform(df)
print("after",df.shape)

before (82140, 13)
after (82140, 2)


In [62]:
from sklearn.ensemble import AdaBoostClassifier
AdaBoost = AdaBoostClassifier(n_estimators=1000,random_state=42)
#n_estimators:樹的數量
AdaBoost.fit(X_train,y_train)
y_pred_Ada = AdaBoost.predict(X_test)
print('Accuracy: %f' % accuracy_score(y_test, y_pred_Ada))
print('f1-score: %f' % f1_score(y_test, y_pred_Ada))

Accuracy: 0.786282
f1-score: 0.653503


In [46]:
ans_pred = AdaBoost.predict(X_ans)
df_sap = pd.DataFrame(ans_pred.astype(int), columns = ['Label'])
df_sap.to_csv('submission_Ada.csv',  index_label = 'Id')

In [29]:
from xgboost import XGBClassifier
#xg=XGBClassifier(random_state=42)
xg = XGBClassifier(colsample_bytree= 0.3, learning_rate=0.01, max_depth= 3, n_estimators=1000)
xg.fit(X_train,y_train)
y_pred_xg = xg.predict(X_test)
print('Accuracy: %f' % accuracy_score(y_test, y_pred_xg))
print('f1-score: %f' % f1_score(y_test, y_pred_xg))

Accuracy: 0.780785
f1-score: 0.639198


In [30]:
ans_pred = xg.predict(X_ans)
df_sap = pd.DataFrame(ans_pred.astype(int), columns = ['Label'])
df_sap.to_csv('submission_xg.csv',  index_label = 'Id')